# Extract Metadata from ITS Base Data from HUE
## Extract Finrep data - POC to extraxt the data

In [1]:

"""Importieren der benötigeten Packages"""
#pip install requests
#pip install requests-kerberos
#pip install pandas
#pip install beautifulsoup4
import pandas as pd
from bs4 import BeautifulSoup
import requests
from requests_kerberos import HTTPKerberosAuth, OPTIONAL
import re
import os  
from pyspark.sql.functions import col
from pyspark.sql.types import DoubleType, DateType
from pyspark.sql.functions import lit
from pyspark.sql import SparkSession
from pyspark_llap import HiveWarehouseSession
from pyspark.sql.types import StructType, StructField, StringType, FloatType
import os
 
################# USER INPUT ########################

hue_databank_name = "its_analyse_test"
table_name = "its_base_data"

#####################################################

hive_table = f"{hue_databank_name}.{table_name}"

# Spark Session with necessary configurations for Hive transactions
spark = SparkSession.builder\
    .appName("hwc-app")\
    .config("spark.security.credentials.hiveserver2.enabled","false")\
    .config("spark.datasource.hive.warehouse.read.via.llap","false")\
    .config("spark.datasource.hive.warehouse.read.jdbc.mode", "client")\
    .config("spark.sql.hive.hiveserver2.jdbc.url","jdbc:hive2://anucdp-mgmt-01.w.oenb.co.at:2181,anucdp-mgmt-02.w.oenb.co.at:2181,anucdp-mgmt-03.w.oenb.co.at:2181/;serviceDiscoveryMode=zooKeeper;zooKeeperNamespace=hiveserver2;trustStoreType=jks;ssl=true")\
    .config("spark.yarn.historyServer.address","https://anucdp-mgmt-02.w.oenb.co.at")\
    .config("spark.sql.hive.hiveserver2.jdbc.url.principal","hive/_HOST@AD.OENB.CO.AT")\
    .config("spark.hadoop.yarn.resourcemanager.principal","hive")\
    .config("spark.kryo.registrator","com.qubole.spark.hiveacid.util.HiveAcidKyroRegistrator")\
    .config("spark.sql.extensions","com.qubole.spark.hiveacid.HiveAcidAutoConvertExtension")\
    .config("spark.jars", "/runtime-addons/spark332-7190-1202-b75-ht9wmb/opt/spark/optional-lib/hive-warehouse-connector-assembly.jar")\
    .getOrCreate()

# Initialize HiveWarehouseSession
hwc = HiveWarehouseSession.session(spark).build()

# Explicitly set the database using SQL
hwc.setDatabase(f'{hue_databank_name}')
hwc.sql(f"USE {hue_databank_name}").show()

hwc.sql("SELECT * FROM its_analyse_test.its_base_data LIMIT 10").show()
#df = hwc.sql("SELECT * FROM its_analyse_test.its_base_data")

Setting spark.hadoop.yarn.resourcemanager.principal to gaubet


++
||
++
++

+---------+------------+-----------------+-----------+--------------------+---------------+-------------------+-------------------+--------+--------------------+--------+--------------+--------------------+--------------+--------------------+--------------+-----------+--------------------+
|datapoint|konzept_code|    taxonomy_code|template_id|      template_label|      module_id| module_gueltig_von| module_gueltig_bis|table_id|          table_name|criteria|x_axis_rc_code|         x_axis_name|y_axis_rc_code|         y_axis_name|z_axis_rc_code|z_axis_name|            idt_hash|
+---------+------------+-----------------+-----------+--------------------+---------------+-------------------+-------------------+--------+--------------------+--------+--------------+--------------------+--------------+--------------------+--------------+-----------+--------------------+
|    10005|ISFIN1240000|FINREP_2016-A-Ind|    F 12.00|Movements in allo...|FINREP_IND_GAAP|2016-12-31 00:00:00|201

In [15]:
# Abfrage für die distinct Werte
#distinct_values = hwc.sql("SELECT DISTINCT module_gueltig_bis FROM its_analyse_test.its_base_data")

#data = distinct_values.collect()  # Liefert eine Liste von Row-Objekten zurück
# Umwandeln in Pandas DataFrame
#pandasDF = pd.DataFrame([row.asDict() for row in data], columns=distinct_values.columns)
#print(pandasDF.head())
#pandasDF
# Zeige die distinct Werte an
#distinct_values.show()
#distinct_values.show(n=distinct_values.count(), truncate=False)

#pandasDF = distinct_values.toPandas()
#print(pandasDF.head())


In [2]:
# Abfrage, um alle Zeilen mit dem spezifischen Wert in der Spalte "module_gueltig_bis" zu erhalten
filtered_rows = hwc.sql("""
    SELECT * 
    FROM its_analyse_test.its_base_data
    WHERE module_gueltig_bis = '3099-12-31 00:00:00'
""")
#filtered_rows.show()
data = filtered_rows.collect()  # Liefert eine Liste von Row-Objekten zurück


# Umwandeln in Pandas DataFrame
pandasDF = pd.DataFrame([row.asDict() for row in data], columns=filtered_rows.columns)
pandasDF.head()

,datapoint,konzept_code,taxonomy_code,template_id,template_label,module_id,module_gueltig_von,module_gueltig_bis,table_id,table_name,criteria,x_axis_rc_code,x_axis_name,y_axis_rc_code,y_axis_name,z_axis_rc_code,z_axis_name,idt_hash
0,10678,ISFIN6080000,FINREP_3.2.1-DP,F 16.04,Gains and losses on financial assets and liabi...,FINREP9_DP,2022-12-31,3099-12-31 00:00:00,F 16.04,None,None,None,None,None,None,None,None,8da304c9629877b7955dca498a7dca5c
1,11133,ISFIN0001590,FINREP_3.2.1,F 14.00,Fair value hierarchy: financial instruments at...,FINREP9,2022-12-31,3099-12-31 00:00:00,F 14.00,Fair value hierarchy: financial instruments at...,None,0060,Level 1,0040,Debt securities,None,None,8a8f74d190a039b2267f136dc0c14533
2,11153,ISFIN0001545,FINREP_3.2.1,F 14.00,Fair value hierarchy: financial instruments at...,FINREP9,2022-12-31,3099-12-31 00:00:00,F 14.00,Fair value hierarchy: financial instruments at...,None,0040,Level 2,0010,Financial assets held for trading,None,None,4cf263150d7bfe47014ab55ab765aed8
3,11476,ISFIN0003518,FINREP_3.2.1,F 20.01,Geographical breakdown of assets by location o...,FINREP9,2022-12-31,3099-12-31 00:00:00,F 20.01,Geographical breakdown of assets by location o...,None,0010,Domestic activities,0240,Derivatives – Hedge accounting,None,None,08b9adabdca474a371f2189df7dc10b7
4,11794,ISFIN0001903,FINREP_3.2.1,F 16.07,Impairment on financial and non-financial assets,FINREP9,2022-12-31,3099-12-31 00:00:00,F 16.07.b,Breakdown of selected statement of profit or l...,None,0040,Accumulated impairment,0070,Subsidiaries,None,None,0d96e9df048a41f05ed3f6c27e07e3af


In [3]:
# Abfrage, um alle Zeilen mit dem spezifischen Wert in der Spalte "module_gueltig_bis" zu erhalten
filtered_rows = hwc.sql("""
    SELECT * 
    FROM its_analyse_test.its_base_data
    WHERE module_gueltig_bis = '3099-12-31 00:00:00'
""")

# Zeige die Ergebnisse an
filtered_rows.show()

# Anzahl der gefilterten Zeilen berechnen
row_count = filtered_rows.count()

# Ausgabe der Anzahl der Zeilen
print(f"Die Anzahl der gefilterten Zeilen: {row_count}")

+---------+----------------+---------------+-----------+--------------------+----------+-------------------+-------------------+---------+--------------------+--------+--------------+--------------------+--------------+--------------------+--------------+-----------+--------------------+
|datapoint|    konzept_code|  taxonomy_code|template_id|      template_label| module_id| module_gueltig_von| module_gueltig_bis| table_id|          table_name|criteria|x_axis_rc_code|         x_axis_name|y_axis_rc_code|         y_axis_name|z_axis_rc_code|z_axis_name|            idt_hash|
+---------+----------------+---------------+-----------+--------------------+----------+-------------------+-------------------+---------+--------------------+--------+--------------+--------------------+--------------+--------------------+--------------+-----------+--------------------+
|    10678|    ISFIN6080000|FINREP_3.2.1-DP|    F 16.04|Gains and losses ...|FINREP9_DP|2022-12-31 00:00:00|3099-12-31 00:00:00|  F 1

Die Anzahl der gefilterten Zeilen: 87341


In [4]:
# Weiter filtern auf 'datapoint' = '112718'
further_filtered_rows = filtered_rows.filter(filtered_rows["datapoint"] == '112718')

# Zeige die gefilterten Zeilen
further_filtered_rows.show()

+---------+------------+---------------+-----------+--------------------+----------+-------------------+-------------------+----------+--------------------+--------+--------------+---------------+--------------+------------+--------------+-----------+--------------------+
|datapoint|konzept_code|  taxonomy_code|template_id|      template_label| module_id| module_gueltig_von| module_gueltig_bis|  table_id|          table_name|criteria|x_axis_rc_code|    x_axis_name|y_axis_rc_code| y_axis_name|z_axis_rc_code|z_axis_name|            idt_hash|
+---------+------------+---------------+-----------+--------------------+----------+-------------------+-------------------+----------+--------------------+--------+--------------+---------------+--------------+------------+--------------+-----------+--------------------+
|   112718|ISFIN0000001|   FINREP_3.2.1|    F 01.01|Balance Sheet Sta...|   FINREP9|2022-12-31 00:00:00|3099-12-31 00:00:00|   F 01.01|Balance Sheet Sta...|    null|          0010|C

In [5]:
# Weiter filtern auf 'datapoint' = '112718' und nur bestimmte Spalten auswählen
further_filtered_rows = (
    filtered_rows
    .filter(filtered_rows["datapoint"] == '112718')
    .select("datapoint", "konzept_code", "x_axis_name", "y_axis_name", "z_axis_name")
)

# Zeige die gefilterten Zeilen
further_filtered_rows.show()

+---------+------------+---------------+------------+-----------+
|datapoint|konzept_code|    x_axis_name| y_axis_name|z_axis_name|
+---------+------------+---------------+------------+-----------+
|   112718|ISFIN0000001|Carrying amount|Cash on hand|       null|
|   112718|ISFIN0000001|           null|        null|       null|
|   112718|ISFIN0000001|           null|        null|       null|
+---------+------------+---------------+------------+-----------+

